In [2]:
!pip install osmnx folium 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 20.7 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 18.3 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 22.0 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.3 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.4 MB/s eta 0:00:0031m15.1 MB/s eta 0:00:01


In [12]:
!pip install geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 7.7 MB/s eta 0:00:00


In [7]:
!pip install geojson

In [3]:
import osmnx as ox

# Download Montreal's street network
montreal_graph = ox.graph_from_place("Montreal, Quebec, Canada", network_type='walk')

# Convert to a geodataframe for manipulation
nodes, edges = ox.graph_to_gdfs(montreal_graph)

KeyboardInterrupt: 

In [4]:
import folium

# Create a map centered on Montreal
map_montreal = folium.Map(location=[45.5017, -73.5673], zoom_start=13)

In [10]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import cKDTree
from geojson import Feature, Point, FeatureCollection, dumps
import numpy as np

crime_data = pd.read_csv('../data/crimes_raw.csv')
# clean the data by removing rows with missing values
crime_data = crime_data.dropna(subset=['LATITUDE', 'LONGITUDE'])

# sample the data to reduce computation time
# crime_data = crime_data.sample(1000)

# Create a geodataframe from the crime data
geometry = [Point(xy) for xy in zip(crime_data['LONGITUDE'], crime_data['LATITUDE'])]
crimes = gpd.GeoDataFrame(crime_data, geometry=geometry)

# create a heatmap from the crime data
from folium import plugins
from folium.plugins import HeatMap

# Create a map centered on Montreal
map_montreal = folium.Map(location=[45.5017, -73.5673], zoom_start=13)

# Add a heatmap to the base map
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in crimes.geometry ]
HeatMap(heat_data, blur=20, radius=10).add_to(map_montreal)

# save heatmap as geojson


# display crimes on folium map
#for i, crime in crimes.iterrows():
 #   folium.Marker([crime['LATITUDE'], crime['LONGITUDE']], popup=crime['CATEGORIE']).add_to(map_montreal)

In [6]:
map_montreal

In [35]:
import pandas as pd
import numpy as np
from geopy.distance import distance
def calculate_crime_score(point, crime_data, radius=0.5):
    point_coords = np.array(point)
    crime_coords = np.array(crime_data.values)
    distances = np.array([distance(point_coords, crime).km for crime in crime_coords[:1000]])
    return np.sum(distances <= radius)

In [33]:
arr_crime_data = crime_data[['LATITUDE','LONGITUDE']]

In [36]:
calculate_crime_score([45.5,-73.6],arr_crime_data)

0

In [39]:
distance((45.5,-73.6),arr_crime_data.values[100])

Distance(3.48408607724284)

In [4]:
place_name = "Montreal, Quebec, Canada"
print("getting graph")
graph = (ox.graph_from_place(place_name, network_type="walk"))
print('got graph')

getting graph
got graph


In [10]:
ox.save_graphml(graph,'../data/montreal.graphml')

In [11]:
g2 = ox.load_graphml('../data/montreal.graphml')

In [7]:
graph

In [8]:
import osmnx as ox
import folium
import networkx as nx

# Step 1: Define the place (Montreal) and load the road network
# Step 2: Simplify the graph
# graph = ox.simplify_graph(graph)

# Step 3: Find two points for demonstration (you can choose any lat/lon)
orig = (45.5017, -73.5673)  # Montreal city center
dest = (45.5175, -73.6568)  # A location west of the city center

# Convert latitude/longitude to nearest nodes in the graph
print('getting nearest nodes')
orig_node = ox.distance.nearest_nodes(graph, orig[1], orig[0])
dest_node = ox.distance.nearest_nodes(graph, dest[1], dest[0])
print('got nearest node')
# Step 4: Calculate the shortest path
shortest_path = nx.shortest_path(graph, orig_node, dest_node, weight="length")
print('got shortest path')
# Step 5: Visualize the path on a folium map
# Create a folium map centered on Montreal
m = folium.Map(location=orig, zoom_start=12)

# Add the nodes and the shortest path to the map
path_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in shortest_path]
folium.PolyLine(path_coords, color="blue", weight=2.5, opacity=1).add_to(m)

# Add markers for origin and destination
folium.Marker(orig, popup="Origin").add_to(m)
folium.Marker(dest, popup="Destination").add_to(m)

# Display the map
m.save("montreal_shortest_path.html")
m


getting nearest nodes
got nearest node
got shortest path
